In [4]:
import pandas as pd
import pymysql

def insert_csv(csv_path: str, table: str,
               host="localhost", port=3306, user="root", password="1234",
               db="tooktook"):
    df = pd.read_csv(csv_path, encoding="utf-8-sig")

    if df.empty:
        print(f"[SKIP] {table}: empty csv -> {csv_path}")
        return

    df = df.replace({True: 1, False: 0})
    df = df.where(pd.notnull(df), None)

    cols = list(df.columns)
    cols_sql = ", ".join(f"{c}" for c in cols)
    placeholders = ", ".join(["%s"] * len(cols))
    sql = f"INSERT INTO {table} ({cols_sql}) VALUES ({placeholders})"

    conn = pymysql.connect(host=host, port=port, user=user, password=password,
                           db=db, charset="utf8", autocommit=False)
    try:
        with conn.cursor() as cur:
            cur.execute("SET NAMES utf8")
            data = list(df.itertuples(index=False, name=None))
            cur.executemany(sql, data)
        conn.commit()
        print(f"[OK] {table}: inserted {len(df)} rows from {csv_path}")
    except Exception as e:
        conn.rollback()
        print(f"[ERR] {table}: {e}")
        raise
    finally:
        conn.close()

insert_csv("./data/consultants.csv", "consultants", password="1234")
insert_csv("./data/customers.csv", "customers", password="1234")
insert_csv("./data/fund_types.csv", "fund_types", password="1234")
insert_csv("./data/documents.csv", "documents", password="1234")
insert_csv("./data/consultations.csv", "consultations", password="1234")
insert_csv("./data/conversation_logs.csv", "conversation_logs", password="1234")
insert_csv("./data/consultation_fund_types.csv", "consultation_fund_types", password="1234")
insert_csv("./data/consultation_documents.csv", "consultation_documents", password="1234")
insert_csv("./data/cls_pred_tomorrow_xgb.csv", "predictions", password="1234")
insert_csv("./data/predictions_tomorrow.csv", "tomorrow_predictions", password="1234")

[OK] consultants: inserted 20 rows from ./data/consultants.csv
[OK] customers: inserted 5000 rows from ./data/customers.csv
[OK] fund_types: inserted 9 rows from ./data/fund_types.csv
[OK] documents: inserted 10 rows from ./data/documents.csv


C:\Users\Admin\AppData\Local\Temp\ipykernel_21308\3305147023.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


[OK] consultations: inserted 30000 rows from ./data/consultations.csv
[OK] conversation_logs: inserted 165159 rows from ./data/conversation_logs.csv
[OK] consultation_fund_types: inserted 43237 rows from ./data/consultation_fund_types.csv
[OK] consultation_documents: inserted 90994 rows from ./data/consultation_documents.csv
[OK] predictions: inserted 9 rows from ./data/cls_pred_tomorrow_xgb.csv
[OK] tomorrow_predictions: inserted 9 rows from ./data/predictions_tomorrow.csv


In [5]:
import pymysql
import pandas as pd
DB = dict(
    host="localhost", port=3306,
    user="root", password="1234",
    database="tooktook", charset="utf8", autocommit=False
)

SQL_DELETE_ALL = "DELETE FROM `statistics`;"

# 1) 전체(ALL) 일자별
SQL_INS_ALL = """
INSERT INTO `statistics`
(`date`,`total_consultations`,`avg_consultation_time`,
 `completed_consultations`,`avg_satisfaction_score`,`category`)
SELECT
  COALESCE(c.`consultation_date`, DATE(c.`start_time`)) AS `date`,
  COUNT(*) AS `total_consultations`,
  ROUND(AVG(CASE WHEN c.`end_time` IS NOT NULL
                 THEN TIMESTAMPDIFF(MINUTE, c.`start_time`, c.`end_time`)
            END)) AS `avg_consultation_time`,
  SUM(c.`end_time` IS NOT NULL) AS `completed_consultations`,
  ROUND(AVG(c.`satisfaction_score`), 2) AS `avg_satisfaction_score`,
  'ALL' AS `category`
FROM `consultations` c
GROUP BY `date`;
"""

# 2) 자금유형별(FUND: ...)
SQL_INS_FUND = """
INSERT INTO `statistics`
(`date`,`total_consultations`,`avg_consultation_time`,
 `completed_consultations`,`avg_satisfaction_score`,`category`)
SELECT
  COALESCE(c.`consultation_date`, DATE(c.`start_time`)) AS `date`,
  COUNT(DISTINCT c.`consultation_id`) AS `total_consultations`,
  ROUND(AVG(CASE WHEN c.`end_time` IS NOT NULL
                 THEN TIMESTAMPDIFF(MINUTE, c.`start_time`, c.`end_time`)
            END)) AS `avg_consultation_time`,
  SUM(c.`end_time` IS NOT NULL) AS `completed_consultations`,
  ROUND(AVG(c.`satisfaction_score`), 2) AS `avg_satisfaction_score`,
  CONCAT('FUND: ', ft.`fund_type_name`) AS `category`
FROM `consultations` c
JOIN `consultation_fund_types` cft ON cft.`consultation_id` = c.`consultation_id`
JOIN `fund_types` ft               ON ft.`fund_type_id`     = cft.`fund_type_id`
GROUP BY `date`, ft.`fund_type_name`;
"""

# 3) 서류별 요구(DOC_REQ: ...)
SQL_INS_DOC_REQ = """
INSERT INTO `statistics`
(`date`,`total_consultations`,`avg_consultation_time`,
 `completed_consultations`,`avg_satisfaction_score`,`category`)
SELECT
  COALESCE(c.`consultation_date`, DATE(c.`start_time`)) AS `date`,
  COUNT(DISTINCT c.`consultation_id`) AS `total_consultations`,
  ROUND(AVG(CASE WHEN c.`end_time` IS NOT NULL
                 THEN TIMESTAMPDIFF(MINUTE, c.`start_time`, c.`end_time`)
            END)) AS `avg_consultation_time`,
  SUM(c.`end_time` IS NOT NULL) AS `completed_consultations`,
  ROUND(AVG(c.`satisfaction_score`), 2) AS `avg_satisfaction_score`,
  CONCAT('DOC_REQ: ', d.`document_name`) AS `category`
FROM `consultations` c
JOIN `consultation_documents` cd ON cd.`consultation_id` = c.`consultation_id`
JOIN `documents` d               ON d.`document_id`      = cd.`document_id`
GROUP BY `date`, d.`document_name`;
"""

# 4) 서류별 제출(DOC_SUB: ...) — documents.is_submitted=1 인 서류만
SQL_INS_DOC_SUB = """
INSERT INTO `statistics`
(`date`,`total_consultations`,`avg_consultation_time`,
 `completed_consultations`,`avg_satisfaction_score`,`category`)
SELECT
  COALESCE(c.`consultation_date`, DATE(c.`start_time`)) AS `date`,
  COUNT(DISTINCT c.`consultation_id`) AS `total_consultations`,
  ROUND(AVG(CASE WHEN c.`end_time` IS NOT NULL
                 THEN TIMESTAMPDIFF(MINUTE, c.`start_time`, c.`end_time`)
            END)) AS `avg_consultation_time`,
  SUM(c.`end_time` IS NOT NULL) AS `completed_consultations`,
  ROUND(AVG(c.`satisfaction_score`), 2) AS `avg_satisfaction_score`,
  CONCAT('DOC_SUB: ', d.`document_name`) AS `category`
FROM `consultations` c
JOIN `consultation_documents` cd ON cd.`consultation_id` = c.`consultation_id`
JOIN `documents` d               ON d.`document_id`      = cd.`document_id`
WHERE d.`is_submitted` = 1
GROUP BY `date`, d.`document_name`;
"""

def main():
    conn = pymysql.connect(**DB)
    try:
        with conn.cursor() as cur:
            cur.execute("SET NAMES utf8")
            # 대시보드용 1회 적재: 비우고 새로 채움
            cur.execute(SQL_DELETE_ALL)

            # ALL / FUND / DOC_REQ / DOC_SUB 순서로 채움
            cur.execute(SQL_INS_ALL)
            cur.execute(SQL_INS_FUND)
            cur.execute(SQL_INS_DOC_REQ)
            cur.execute(SQL_INS_DOC_SUB)

        conn.commit()
        print("[OK] statistics inserted.")

        # 확인: 최근 30행만 보기 (pandas)
        df = pd.read_sql(
            "SELECT * FROM `statistics` ORDER BY `date` DESC, `category` LIMIT 30;",
            conn
        )
        print(df)
    except Exception as e:
        conn.rollback()
        raise
    finally:
        conn.close()

if __name__ == "__main__":
    main()

[OK] statistics inserted.
    statistic_id        date  total_consultations  avg_consultation_time  \
0             53  2025-08-25                   72                     23   
1           3983  2025-08-25                   19                     25   
2           3984  2025-08-25                   23                     24   
3           3985  2025-08-25                   26                     24   
4           3986  2025-08-25                   19                     21   
5           3987  2025-08-25                   22                     23   
6           3988  2025-08-25                   21                     24   
7           3989  2025-08-25                   20                     20   
8           3990  2025-08-25                   19                     22   
9           3991  2025-08-25                   17                     23   
10          3992  2025-08-25                   22                     22   
11          5358  2025-08-25                   23             

C:\Users\Admin\AppData\Local\Temp\ipykernel_21308\3773195915.py:108: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
